In [1]:
import numpy as np
import torch
from itertools import product
from collections import defaultdict
from Agents import *
from Config import Config

In [2]:
config = Config()

In [3]:
config

Config(I=3, v_bar=1, sigma_v=1, Np=31, Nv=10, Nx=15, iota=0.1, rho=0.95, alpha=0.01, beta=1e-05, xi=500, theta=0.1, Tm=10000, u_bar=0, sigma_u=1)